In [76]:
from graphviz import Digraph
import networkx as nx
import random
import pandas as pd

def generate_random_color():
    return '#' + ''.join(random.choice('0123456789ABCDEF') for _ in range(6))

def determine_text_color(hex_color):
    red = int(hex_color[1:3], 16)
    green = int(hex_color[3:5], 16)
    blue = int(hex_color[5:7], 16)
    
    brightness = (red * 299 + green * 587 + blue * 114) / 1000
    
    return '#000000' if brightness > 128 else '#FFFFFF'

diagram = Digraph(
    node_attr={'shape': 'box'},
    graph_attr={
        'ordering': 'out',
        'splines': 'ortho',
        'dpi': '300',
        'ratio': 'fill',
        'ranksep': '1.0 equally'
    },  
)

dataframe_activities = pd.read_csv('./lista-atividades.csv', nrows=51)

subject_dictionary = {}

color_mapping = {
    'BIQ': '#fac7b4', 'FAS': '#49cccc', 'FIB': '#aee1d3', 'MAT': '#e6b250', 
    'MOF': '#83ad92', 'QUI': '#fca89d', 'BIG': '#afc7b9', 'PFA': '#a892ff', 
    'MIC': '#dd7fdc', 'PAG': '#ffdef4', 'PAR': '#5d77ce', 'BOT': '#d1e751', 
    'EST': '#998b82', 'FAR': '#ffe1c9', 'ACT': '#1E6D2D', 'ALM': '#92a6ef', 
    'FAF': '#eecb68'
}

max_subjects = 8
num_electives = 0
edges = []

for index in range(0, 10):
    index_bounds = [
        [0,7], [8,14], [15,19], [20,25], [26,31], 
        [32,37], [38,44], [45,46], [47,48], [49,50]
    ]

    lower_bound = index_bounds[index][0]
    upper_bound = index_bounds[index][1]

    with diagram.subgraph() as subgraph:
        subgraph.attr(rank=('same' if index != 9 else 'sink'))
        subjects = []
        for row_index, row in dataframe_activities.iterrows():
            if row_index < lower_bound:
                continue
            elif row_index > upper_bound:
                continue

            print(index, row_index, row.Nome)

            subject_code = row.Nome.split(' - ')[1]
            subject_name = row.Nome.split(' - ')[2]

            subject_dictionary[subject_code] = subject_name
            subjects.append(subject_code)

            prefix = subject_code[:3]
            if prefix not in color_mapping:
                color_mapping[prefix] = generate_random_color()

        for subject_code in subjects:
            prefix = subject_code[:3]
            text_color = determine_text_color(color_mapping[prefix])
            subgraph.node(
                subject_dictionary[subject_code], 
                subject_dictionary[subject_code], 
                style='filled', 
                fillcolor=color_mapping[prefix], 
                fontcolor=text_color, 
                fontweight='bold', 
                fontname='Helvetica-Bold'
            )
  
for row_index, row in dataframe_activities.iterrows():
    subject_code = row.Nome.split(' - ')[1]
    subject_name = row.Nome.split(' - ')[2]

    prerequisites = row.Atividades.split(', ')
    for prerequisite in prerequisites:
        if prerequisite != '-':
            prerequisite_code = prerequisite.replace(',', '')
            try:
                prerequisite_name = subject_dictionary[prerequisite_code]
                diagram.edge(prerequisite_name, subject_name)
                edges.append([prerequisite_name, subject_name])
            except KeyError:
                pass

print(color_mapping)

diagram.render(
    'diagrama_farmacia',
    format='png',
    view=True,
)



0 0  - BIQ050 - BIOQUIMICA CELULAR F
0 1  - FAS007 - FARMACIA E SOCIEDADE
0 2  - FIB034 - BIOFISICA B
0 3  - MAT130 - MATEMATICA
0 4  - MOF009 - ANATOMIA HUMANA BASICA
0 5  - MOF142 - CITOLOGIA E HISTOLOGIA F
0 6  - QUI203 - QUIMICA GERAL F
0 7 DIG - QUI204 - QUIMICA GERAL EXPERIMENTAL F
1 8 DIG - QUI293 - ELEMENTOS DE FÍSICO-QUÍMICA
1 9  - BIG157 - GENÉTICA F
1 10  - BIQ602 - IMUNOLOGIA BASICA
1 11  - FIB035 - FISIOLOGIA F
1 12  - PFA131 - FUNDAMENTOS DE CIÊNCIAS FARMACÊUTICAS
1 13  - QUI291 - QUÍMICA INORGÂNICA F
1 14  - QUI292 - QUÍMICA INORGÂNICA EXPERIMENTAL F
2 15  - MIC030 - MICROBIOLOGIA BÁSICA F
2 16  - PAG011 - PATOLOGIA GERAL F
2 17  - PAR026 - PARASITOLOGIA HUMANA F
2 18  - QUI207 - QUIMICA ORGANICA I F
2 19 DIG - QUI208 - QUIMICA ANALITICA F
3 20  - BOT035 - BOTÂNICA F
3 21  - EST083 - BIOESTATISTICA BASICA F
3 22  - FAR024 - FARMACOLOGIA BASICA F
3 23  - QUI210 - QUIMICA ORGANICA EXPERIMENTAL F
3 24  - QUI294 - QUÍMICA ORGÂNICA II F
3 25 DIG - QUI295 - QUÍMICA ANALÍTICA I

In [77]:
nx_g = nx.DiGraph()

nx_g.add_edges_from(edges)

print(nx_g.degree)

[('MATEMATICA', 1), ('ELEMENTOS DE FÍSICO-QUÍMICA', 6), ('QUIMICA GERAL F', 4), ('QUIMICA GERAL EXPERIMENTAL F', 4), ('BIOQUIMICA CELULAR F', 3), ('IMUNOLOGIA BASICA', 3), ('CITOLOGIA E HISTOLOGIA F', 2), ('FISIOLOGIA F', 6), ('BIOFISICA B', 2), ('ANATOMIA HUMANA BASICA', 1), ('QUÍMICA INORGÂNICA F', 4), ('QUÍMICA INORGÂNICA EXPERIMENTAL F', 3), ('MICROBIOLOGIA BÁSICA F', 2), ('PATOLOGIA GERAL F', 3), ('PARASITOLOGIA HUMANA F', 3), ('QUIMICA ORGANICA I F', 4), ('QUIMICA ANALITICA F', 4), ('FARMACOLOGIA BASICA F', 6), ('QUIMICA ORGANICA EXPERIMENTAL F', 3), ('QUÍMICA ORGÂNICA II F', 3), ('QUÍMICA ANALÍTICA INSTRUMENTAL F', 2), ('ESTUDO C. L. DOENCAS HUMANAS A', 3), ('BROMATOLOGIA', 1), ('FARMACIA E SOCIEDADE', 1), ('EPIDEMIOLOGIA', 5), ('BIOESTATISTICA BASICA F', 1), ('FARMACOCINETICA', 4), ('QUIMICA FARMACEUTICA E MEDICINAL I', 3), ('BOTÂNICA F', 1), ('FARMACOGNOSIA I', 4), ('TOXICOLOGIA GERAL', 1), ('OPERACOES UNITARIAS NA AREA FARMACEUTICA', 1), ('POLITICAS DE SAUDE', 1), ('FARMACOGN